In [1]:
# The dataset used here is the famous "20 Newsgroup" data set.
# The data set is in-built in scikit
# Loading the Data Set
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)

In [2]:
# Check the target names (categories)
twenty_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [3]:
twenty_train.target

array([7, 4, 4, ..., 3, 1, 8])

In [4]:
# prints first line of the first data file
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu


In [18]:
# Extracting features from text files
'''
Text files are actually series of words(ordered). In order to run ML algorithms we need to convert the text files into numerical feature vectors. We will use "bag of words" model.
Each unique word in our dictionary will correspond to a feature.
'''
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
'''
Using count_vect.fit_transform, we are learning the vocabulary dictionary and it returns a Document-Term matrix
'''
X_train_counts.shape


(11314, 130107)

In [20]:
'''
TF (Term Frequency) helps in avoiding the issue with giving more weight to longer documents than shorter documents. count(words) / Total words (in each document)
TF-IDF even reduces the weightage of more common words in documents (e.g., the, is an etc)
'''
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 130107)

In [21]:
# Running ML algorithms
'''
There are various algorithms which can be used for text classification. 
We start with 'Naive Bayes'
The below code will train the NB classifier on the training data we provided
'''

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf,twenty_train.target)

In [13]:
twenty_train.target

array([7, 4, 4, ..., 3, 1, 8])

In [6]:
# Building a pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                     ])
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

In [7]:
# Performance of NB Classifier
# Using the Test Set
import numpy as np
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)
predicted = text_clf.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

0.7738980350504514

In [8]:
# Support Vector Machine
from sklearn.linear_model import SGDClassifier

text_clf_svm = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge',penalty='l2',alpha=1e-3,random_state=42)),
                         ])
_ = text_clf_svm.fit(twenty_train.data, twenty_train.target)
predicted_svm = text_clf_svm.predict(twenty_test.data)
np.mean(predicted_svm == twenty_test.target)

0.8240839086563994

In [31]:
# Grid Search
'''
Almost all the classifiers will have various parameters which can be tuned to obtain optimal performance.
Here we are creating a list of parameters for which we would like to do performance tuning.
The parameter names start with the classifier name
'''
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range':[(1,1),(1,2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2,1e-3),
              }

In [39]:
# Here we create an instance of the grid search by passing the classifier, parameters and n_jobs = -1 which tells to use multiple cores from user machine
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

In [40]:
gs_clf.best_score_
gs_clf.best_params_

{'clf__alpha': 0.01, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [41]:
'''
The accuracy has now increased to ~83.62% for the NB Classifier
The corresponding parameters are {'clf__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}
'''
predicted = gs_clf.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

0.8300584174190122

In [37]:
# We get improved accuracy ~83.51% from ~82.40% for SVM classifier.
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
                  'tfidf__use_idf': (True, False),
                  'clf-svm__alpha': (1e-2, 1e-3),
                  }
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(twenty_train.data, twenty_train.target)
gs_clf_svm.best_score_
gs_clf_svm.best_params_

predicted_svm = gs_clf_svm.predict(twenty_test.data)
np.mean(predicted_svm == twenty_test.target)

0.8351035581518853

In [38]:
# Now removing the stop words from the data. This should be done when we know that Stop Words are not useful for the underlying problem.
# Testing NB classifier by removing the Stop Words
# Building a pipeline

text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                     ])

text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

predicted = text_clf.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

0.8169144981412639

In [43]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer,self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect),
                             ('tfidf', TfidfTransformer()),
                             ('mnb', MultinomialNB(fit_prior=False)),
                              ])
text_mnb_stemmed = text_mnb_stemmed.fit(twenty_train.data, twenty_train.target)
predicted_mnb_stemmed = text_mnb_stemmed.predict(twenty_test.data)
np.mean(predicted_mnb_stemmed == twenty_test.target)

0.8167817312798725